# Artificial Neural Network Play 2: Simple Neural Networks

Part one can be found [here](http://nbviewer.ipython.org/github/Ogaday/ml-play/blob/master/ANN-Play%2001%20Perceptrons.ipynb) and is part of a series on github [here](https://github.com/Ogaday/ml-play). Sources are inlcuded in the first notebook.

This chapter will be focussed on building and activating simple neural networks.

## A simple neural network

Neural networks can be characterized graphically: each neuron or node is represented by a vertex and each axon by an edge. There are several strategies for defining graphs. Commonly, graphs are defined by a triple: $(V, E, \varepsilon)$ where $V$ is the set of vertices, $E$ is the set of edges and $\varepsilon$ is the endpoint map $\varepsilon:E\to P_1\{V\}\cup P_2\{V\}$. (A mapping of edges to the union of the power set of one and the power set of two of the set of vertices). In this case, the graph is weighted, so we introduce a metric map $\mu$ such that $\mu: E \to \mathbb R$, where $\mathbb R$ is the set of real values and the mapping denotes of the weight on each edge. Furthermore, this graph should be loop free, so in fact we can simplify $\varepsilon$ to $\varepsilon:E\to P_2\{V\}$ and as this graph is directed, order of the elements of $p_i \in P_2\{V\}$ matters.

However, there is a representation that is better suited for translation to a data structure:  the adjacency matrix. For a weighted graph, this is simply defined by $A_{ij} = w_{ij}$ where $w_{ij}$ is the weight of the edge between the vertices $i$ and $j$. This is a much more direct way of representing a graph digitally.

I implemented a quick graphical representation of an artificial nueral network in order to go through Carlos Gershenson's [tutorial](https://datajobs.com/data-science-repo/Neural-Net-%5BCarlos-Gershenson%5D.pdf).

In an adjacency matrix, the $i, j^{th}$ entry represents the weight on the edge between the $i$ and $j$. I use a negative value to represent no edge. **Lemma**: There is no cost to traverse nowhere, so the entry $(i,i):= 0$ (this could however, maybe more practically be -1 in a neural network I believe my implementation is agnostic to it.). **Corollary**: In order represent multiple edges between vertices, it is necessary to have instead the $i, j^{th}$ entry as an array of the edges between the two vertices. This is not a requirement for dealing with neural networks, so we will stick with the original implementation. **Corollary**: This form is most suited to representing directed graphs, which is exactly what we need. Instead, we could impose symetry on the adjacency matrix along the leading diagonal, so the cost of traversing between two vertices is equivalent. This, however, will not be necessary. **Finally**: As we are representing the network as a graph, I simulate the inputs and outputs with additional vertices. That is, despite frequent illustrations of these neworks with inputs and outputs having no terminating feature, each edge starts or terminates with a node in my represenation.

Let's try to define the network necessary to describe exercise $3.1$ in the tutorial. Gershenson describes a network with an input layer containing two neurons and an output layer containing two neurons. Here are some ascii illustrations:
```

o    o
|    |
|    |
o    o
|\  /|
| \/ |
| /\ |
|/  \|
o    o
|    |
|    |
o    o

o  o
|  |
o  o
|\/|
|/\|
o  o
|  |
o  o

--->
0  2     4  6
o--o-----o--o
    \   /
     \ /
      x
     / \
    /   \
o--o-----o--o
1  3     5  7
```

Evidently, I already have a problem... What if the weights want to be negative or $0$? Do unconnected nodes as ```None```? Keep an independent list of edges? But then I'm back to the tripple represntation of $(V,E, \varepsilon)$ and that's more complicated. Maybe I do go back to that representation and implement a vertex object? Alternatively... look back at my own suggestion and have each $i,j^{th}$ member represented as a list: empty means no connection edge. Seems wasteful, but could be doable. Alternatively... just have a ```None``` where there is no connection. That works too. Seems antipatterny. Using the None solution below. Will also look into adjency lists, might be more practical. Using empty lists is also perhaps more pythonic/less dagerous, but I thought the memory costs would be higher, and less easy to convert to numpy arrays.

In [1]:
from ann import ANN

In [2]:
# Initial graph with all edges weighted as 1.
G=ANN([[None,None,   1,None,None,None,None,None],
       [None,None,None,   1,None,None,None,None],
       [None,None,None,None,   1,   1,None,None],
       [None,None,None,None,   1,   1,None,None],
       [None,None,None,None,None,None,   1,None],
       [None,None,None,None,None,None,None,   1],
       [None,None,None,None,None,None,None,None],
       [None,None,None,None,None,None,None,None]], sum)

Now that we have out Neural Network, characterised by a graph and represented by an adjacency matrix, we can start to look at activating it. My implementation looks at those vertices which are not at the head of an edge connected to any other vertex and then uses a recursive call to call the function (in this case ```sum```) on the dot product of the weights and the inputs for that vertex, continuing until at the input nodes.

In [3]:
# Input vector of ones:
input_vectors = ones, floats, one_neg, zeros = [1,1], [0.3, 0.7], [0.5, -0.5], [0,0]
# outputs = ones_in, floats_in, one_neg_in, zeros_in = 0,0,0,0
descriptions = ["Ones in", "Floats in", "One negative", "Zeroes in"]

ones_in = G.feedforward(ones)
floats_in = G.feedforward(floats)
one_neg_in = G.feedforward(one_neg)
zeros_in = G.feedforward(zeros)

outputs = [ones_in, floats_in, one_neg_in, zeros_in]

In [4]:
for i, o, d in zip(input_vectors, outputs, descriptions):
    print('For {} : {} the output was {}'.format(d.lower(), i, o))

For ones in : [1, 1] the output was {6: 2, 7: 2}
For floats in : [0.3, 0.7] the output was {6: 1.0, 7: 1.0}
For one negative : [0.5, -0.5] the output was {6: 0.0, 7: 0.0}
For zeroes in : [0, 0] the output was {6: 0, 7: 0}


In [5]:
# Try the values from the tutorial:
vals = [[1,1],[1,0],[0,1],[0,0],[-1,1],[-1,-1]]
for val in vals:
    print("Input: {}\nOutput: {}\n".format(val, G.feedforward(val)))

Input: [1, 1]
Output: {6: 2, 7: 2}

Input: [1, 0]
Output: {6: 1, 7: 1}

Input: [0, 1]
Output: {6: 1, 7: 1}

Input: [0, 0]
Output: {6: 0, 7: 0}

Input: [-1, 1]
Output: {6: 0, 7: 0}

Input: [-1, -1]
Output: {6: -2, 7: -2}



In [15]:
# Let's try changing the weights:
edges = []
for i in range(8):
    for j in range(8):
        if G.matrix[i][j] != None:
            edges.append((i,j))

import random
random.seed(0)

layouts = {}
for i in range(10):
    layouts[i] = []
    for edge in edges:
        layouts[i].append(random.choice([-0.5, 0, 0.5]))

In [17]:
print(layouts)
for l, choices in layouts.items():
    for k, e in enumerate(edges):
        i, j = e
        G.matrix[i][j] = choices[k]
    for g in G.matrix:
        print(g)
    for val in vals:
        print("Input: {}\nOutput: {}\n".format(val, G.feedforward(val)))

{0: [0, 0, -0.5, 0, 0.5, 0, 0, 0], 1: [0, 0, 0.5, -0.5, 0.5, -0.5, 0, -0.5], 2: [-0.5, 0.5, 0, 0.5, 0.5, 0.5, -0.5, 0], 3: [-0.5, 0.5, -0.5, 0.5, 0, 0, 0.5, -0.5], 4: [0, 0, 0, 0.5, 0.5, -0.5, 0.5, 0], 5: [0, 0.5, 0, -0.5, 0.5, -0.5, -0.5, 0.5], 6: [0, 0.5, 0.5, 0.5, -0.5, 0.5, 0, 0], 7: [-0.5, 0.5, 0, 0.5, -0.5, -0.5, 0.5, -0.5], 8: [-0.5, -0.5, 0.5, 0, -0.5, -0.5, 0, 0.5], 9: [0, -0.5, 0, 0.5, 0, 0.5, -0.5, 0.5]}
[None, None, 0, None, None, None, None, None]
[None, None, None, 0, None, None, None, None]
[None, None, None, None, -0.5, 0, None, None]
[None, None, None, None, 0.5, 0, None, None]
[None, None, None, None, None, None, 0, None]
[None, None, None, None, None, None, None, 0]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
Input: [1, 1]
Output: {6: 0.0, 7: 0}

Input: [1, 0]
Output: {6: 0.0, 7: 0}

Input: [0, 1]
Output: {6: 0.0, 7: 0}

Input: [0, 0]
Output: {6: 0.0, 7: 0}

Input: [-1, 1]
Output: {6: 0.0, 7: 0}

Input: [-1, -1]
O

Network so that the inputs come out in reverse order. ie:
```
        --->
        0  2     4  6
0.3 --> o--o-----o--o --> 0.7
            \   /
             \ /
              x
             / \
            /   \
0.7 --> o--o-----o--o --> 0.3
        1  3     5  7
```

In [20]:
# Try:
p = [(0,2),(1,3),(2,5),(3,4),(4,6),(5,7)]
for e in edges:
    i, j = e
    if e in p:
        G.matrix[i][j] = 1
    else:
        G.matrix[i][j] = 0
for g in G.matrix:
    print(g)
for val in vals:
    print("Input: {}\nOutput: {}\n".format(val, G.feedforward(val)))
print("Input: {}\nOutput: {}\n".format([0.3, 0.7], G.feedforward([0.3, 0.7])))

[None, None, 1, None, None, None, None, None]
[None, None, None, 1, None, None, None, None]
[None, None, None, None, 0, 1, None, None]
[None, None, None, None, 1, 0, None, None]
[None, None, None, None, None, None, 1, None]
[None, None, None, None, None, None, None, 1]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
Input: [1, 1]
Output: {6: 1, 7: 1}

Input: [1, 0]
Output: {6: 0, 7: 1}

Input: [0, 1]
Output: {6: 1, 7: 0}

Input: [0, 0]
Output: {6: 0, 7: 0}

Input: [-1, 1]
Output: {6: 1, 7: -1}

Input: [-1, -1]
Output: {6: -1, 7: -1}

Input: [0.3, 0.7]
Output: {6: 0.7, 7: 0.3}



Network so that the inputs come out double in value. ie:
```
        --->
        0  2     4  6
0.3 --> o--o-----o--o --> 0.6
            \   /
             \ /
              x
             / \
            /   \
0.7 --> o--o-----o--o --> 1.4
        1  3     5  7
```

In [21]:
# Try:
p = [(0,2),(1,3),(4,6),(5,7)]
q = [(2,4),(3,5)]
for e in edges:
    i, j = e
    if e in p:
        G.matrix[i][j] = 1
    elif e in q:
        G.matrix[i][j] = 2
    else:
        G.matrix[i][j] = 0
for g in G.matrix:
    print(g)
for val in vals:
    print("Input: {}\nOutput: {}\n".format(val, G.feedforward(val)))
print("Input: {}\nOutput: {}\n".format([0.3, 0.7], G.feedforward([0.3, 0.7])))

[None, None, 1, None, None, None, None, None]
[None, None, None, 1, None, None, None, None]
[None, None, None, None, 2, 0, None, None]
[None, None, None, None, 0, 2, None, None]
[None, None, None, None, None, None, 1, None]
[None, None, None, None, None, None, None, 1]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
Input: [1, 1]
Output: {6: 2, 7: 2}

Input: [1, 0]
Output: {6: 2, 7: 0}

Input: [0, 1]
Output: {6: 0, 7: 2}

Input: [0, 0]
Output: {6: 0, 7: 0}

Input: [-1, 1]
Output: {6: -2, 7: 2}

Input: [-1, -1]
Output: {6: -2, 7: -2}

Input: [0.3, 0.7]
Output: {6: 0.6, 7: 1.4}



Network such that if the sum of the inputs is greater than a threshhold $T$ then the neuron fires with a value of $1$, otherwise, $0$.

In [41]:
def threshmaker(T):
    return lambda x: 1 if sum(x) > T else 0

threshhold1 =threshmaker(1)
threshhold0_5 =threshmaker(0.5)

Gthresh=G=ANN([[None,None,   1,None,None,None,None,None],
       [None,None,None,   1,None,None,None,None],
       [None,None,None,None,   1,   1,None,None],
       [None,None,None,None,   1,   1,None,None],
       [None,None,None,None,None,None,   1,None],
       [None,None,None,None,None,None,None,   1],
       [None,None,None,None,None,None,None,None],
       [None,None,None,None,None,None,None,None]], threshhold1)

In [42]:
T = threshmaker(5)

print(T([1,4.1]))

1


In [43]:
for val in vals:
    val = [2*v for v in val]
    print("Input: {}\nOutput: {}\n".format(val, Gthresh.feedforward(val)))
print("Input: {}\nOutput: {}\n".format([0.3, 0.7], Gthresh.feedforward([0.3, 0.7])))

Input: [2, 2]
Output: {6: 1, 7: 1}

Input: [2, 0]
Output: {6: 0, 7: 0}

Input: [0, 2]
Output: {6: 0, 7: 0}

Input: [0, 0]
Output: {6: 0, 7: 0}

Input: [-2, 2]
Output: {6: 0, 7: 0}

Input: [-2, -2]
Output: {6: 0, 7: 0}

Input: [0.3, 0.7]
Output: {6: 0, 7: 0}



In [55]:
Gthresh=ANN([[None,None,   10,None,None,None,None,None],
       [None,None,None,   10,None,None,None,None],
       [None,None,None,None,   10,   10,None,None],
       [None,None,None,None,   10,   10,None,None],
       [None,None,None,None,None,None,   10,None],
       [None,None,None,None,None,None,None,   10],
       [None,None,None,None,None,None,None,None],
       [None,None,None,None,None,None,None,None]], threshmaker(-10))

choices = layouts[7]
for k, e in enumerate(edges):
    i, j = e
    Gthresh.matrix[i][j] = choices[k]
for g in Gthresh.matrix:
    print(g)
for val in vals:
    print("Input: {}\nOutput: {}\n".format(val, Gthresh.feedforward(val)))

[None, None, -0.5, None, None, None, None, None]
[None, None, None, 0.5, None, None, None, None]
[None, None, None, None, 0, 0.5, None, None]
[None, None, None, None, -0.5, -0.5, None, None]
[None, None, None, None, None, None, 0.5, None]
[None, None, None, None, None, None, None, -0.5]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
Input: [1, 1]
Output: {6: 0.5, 7: -0.5}

Input: [1, 0]
Output: {6: 0.5, 7: -0.5}

Input: [0, 1]
Output: {6: 0.5, 7: -0.5}

Input: [0, 0]
Output: {6: 0.5, 7: -0.5}

Input: [-1, 1]
Output: {6: 0.5, 7: -0.5}

Input: [-1, -1]
Output: {6: 0.5, 7: -0.5}



Network which implements the logical "and" function on one output and logical "or" function on the other output.

In [67]:
inputs=[[1,1],[1,0],[0,1],[0,0]]
AND=[1,0,0,0]
OR=[1,1,1,0]

Glogic=ANN([[None,None,   1,None,None,None,None,None],
       [None,None,None,   1,None,None,None,None],
       [None,None,None,None,   0.5,   1,None,None],
       [None,None,None,None,   0.5,   1,None,None],
       [None,None,None,None,None,None,   1,None],
       [None,None,None,None,None,None,None,   1],
       [None,None,None,None,None,None,None,None],
       [None,None,None,None,None,None,None,None]], threshmaker(0.9))

for j,i in enumerate(inputs):
    print("Inputs:  {}\nResults: {}\nAND, OR:  {}, {}".format(i, [Glogic.feedforward(i)[j] for j in range(6,8)], AND[j], OR[j]))

Inputs:  [1, 1]
Results: [1, 1]
AND, OR:  1, 1
Inputs:  [1, 0]
Results: [0, 1]
AND, OR:  0, 1
Inputs:  [0, 1]
Results: [0, 1]
AND, OR:  0, 1
Inputs:  [0, 0]
Results: [0, 0]
AND, OR:  0, 0


In [ ]:
import sys
print(sys.version)

In [ ]:
%pylab inline
import numpy as np

In [ ]:
figsize(10,8)
plot([i for i in range(10)], [np.random.randint(-10,10) for i in range(10)])
x = np.linspace(0,9)
plot(x, [np.random.randint(-10,10) for xx in x], "r")
plot(x, 7*np.sin(x), "y")